In [50]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# Define the stock symbol and download historical data from Yahoo Finance
stock_symbol = "ZOMATO.NS"  # Change this to the desired stock symbol
start_date = "2023-01-01"  # YYYY-MM-DD
end_date = "2023-08-31"   # YYYY-MM-DD
df = yf.download(stock_symbol, start=start_date, end=end_date)

# Feature engineering (example features)
df['PriceChange'] = df['Close'] - df['Open']
df['PriceChangePercentage'] = (df['Close'] - df['Open']) / df['Open'] * 100

# Create a binary target variable for trend prediction
df['Trend'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)

# Drop rows with missing values
df.dropna(inplace=True)

# Define the features (X) and the target variable (y)
X = df[['Open', 'High', 'Low', 'Close', 'Volume', 'PriceChange', 'PriceChangePercentage']]
y = df['Trend']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features (important for some algorithms)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a Logistic Regression model
model = LogisticRegression(random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', classification_rep)

# Predict the future trend for a new data point (example)
# Replace these values with the actual feature values of your new data point
new_open = 145.0
new_high = 150.0
new_low = 142.0
new_close = 148.0
new_volume = 1500000
new_price_change = new_close - new_open
new_price_change_percentage = (new_close - new_open) / new_open * 100

new_data = np.array([new_open, new_high, new_low, new_close, new_volume, new_price_change, new_price_change_percentage]).reshape(1, -1)
new_data = scaler.transform(new_data)
future_trend = model.predict(new_data)
print(f'Predicted Future Trend: {"Up" if future_trend[0] == 1 else "Down"}')

# Create a candlestick chart to visualize historical stock prices
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.update_layout(
    title=f'Candlestick Chart for {stock_symbol}',
    xaxis_title='Date',
    yaxis_title='Price',
)

fig.show()


[*********************100%***********************]  1 of 1 completed
Accuracy: 0.48484848484848486
Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.27      0.32        15
           1       0.52      0.67      0.59        18

    accuracy                           0.48        33
   macro avg       0.46      0.47      0.45        33
weighted avg       0.47      0.48      0.46        33

Predicted Future Trend: Up
